In [1]:
import numpy as np
import pandas as pd

In [2]:
import tensorflow as tf
from tensorflow import keras

In [22]:
def generate_time_series(batch_size, n_steps):
    freq1, freq2, offset1, offset2 = np.random.rand(4, batch_size, n_steps)
    time = np.linspace(0, 1, n_steps)
    series = 0.5 * np.sin((time - offset1) * (freq1 * 10 + 10))
    series += 0.2 * np.sin((time - offset2) * (freq2 * 10 + 10))
    series += 0.1 * (np.random.rand(batch_size, n_steps) - 0.5)
    return series[..., np.newaxis].astype(np.float32)

In [27]:
series = generate_time_series(50, 10)

In [28]:
series.shape

(50, 10, 1)

In [37]:
n_steps = 50
series = generate_time_series(10000, n_steps + 1)
X_train, y_train = series[:7000, :n_steps], series[:7000, -1]
X_valid, y_valid = series[7000:9000, :n_steps], series[7000:9000, -1]
X_test, y_test = series[9000:, :n_steps], series[9000:, -1]

In [38]:
X_train.shape, y_train.shape

((7000, 50, 1), (7000, 1))

In [41]:
series = generate_time_series(10000, n_steps + 10) 
X_train, y_train = series[:7000, :n_steps], series[:7000, -10:, 0] 
X_valid, y_valid = series[7000:9000, :n_steps], series[7000:9000, -10:, 0] 
X_test, y_test = series[9000:, :n_steps], series[9000:, -10:, 0]

In [42]:
X_train.shape, y_train.shape

((7000, 50, 1), (7000, 10))

In [43]:
Y = np.empty((10000, n_steps, 10)) # each target is a sequence of 10D vectors 
for step_ahead in range(1, 10 + 1): 
    Y[:, :, step_ahead - 1] = series[:, step_ahead:step_ahead + n_steps, 0] 
Y_train = Y[:7000] 
Y_valid = Y[7000: 9000] 
Y_test = Y[9000:]

In [44]:
Y_train.shape

(7000, 50, 10)

In [45]:
Y_train[:, 3::2].shape

(7000, 24, 10)

### Batch Chorals Model

In [50]:
import os
import tarfile

if "RNN" in os.path.abspath(os.curdir): os.chdir("..")
BASE_DIR = os.path.abspath(os.curdir)
DATA_PATH = os.path.join(BASE_DIR, "datasets", "rnn")

# fetch data
def extract_data(data_path=DATA_PATH):
    os.makedirs(data_path, exist_ok=True)
    tgz_path = os.path.join(data_path, "jsb_chorales.tgz")
    data_tgz = tarfile.open(tgz_path)
    data_tgz.extractall(path=data_path)
    data_tgz.close()
    
# extract_data()